In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import pyodbc
con = pyodbc.connect('DSN=ISDW')

In [11]:
## TS measurement wafer map pull
waferids = """
'6570065LSEH0', 
'65700B9USEB1', 
'6A06G155SOF0', 
'6A06G308SOA7', 
'6A06R286SOD7', 
'6A078256SOF6', 
'6570066GSEF0', 
'6A03T405SOF2', 
'6A05T283SOD5', 
'6A066057SOA5', 
'6A066083SOE4', 
'6A069273SOG6', 
'6A06G265SOB2', 
'6A06H100SOF7', 
'6A06H164SOF5', 
'6A06I141SOG3', 
'6A06J080SOG5', 
'6A06K153SOF3', 
'6A06L145SOC3', 
'6A06N056SOF3', 
'6A070220SOA5', 
'6A075071SOB4', 
'6A076153SOA4', 
'6A078022SOG4', 
'65700A2ASED4', 
'65700A3ISEF1', 
'65700J29SEF7', 
'65700K3CSEA3', 
'65700K3FSEF6', 
'65700M2ISEE7', 
'6A05D273SOA4', 
'6A05T298SOH2', 
'6A066210SOC6', 
'6A066211SOH0', 
'6A06A159SOF0', 
'6A06A173SOD2', 
'6A06A198SOE0', 
'6A06E256SOA2', 
'6A06H271SOD7', 
'6A06I138SOC1', 
'6A06L027SOC6', 
'6A06M131SOD3', 
'6A06M134SOB3', 
'6A06U126SOB0', 
'6A06W137SOD1', 
'6A077091SOD7', 
'6A077358SOF5', 
'65603S2ISEG2', 
'6570074FSEF5', 
'65700A2FSEC5', 
'65700A9USEC0', 
'65700AADSEC5', 
'65700J06SEG2', 
'6A064216SOF3', 
'6A066054SOC5', 
'6A068053SOG4', 
'6A068207SOG5', 
'6A069045SOD4', 
'6A069046SOA3', 
'6A06G156SOB7', 
'6A06G177SOA3', 
'6A06G224SOB1', 
'6A06H072SOC4', 
'6A06K081SOD7', 
'6A06L073SOA7', 
'6A06L198SOA6', 
'6A06N291SOA3', 
'6A06X054SOA5', 
'6570074ESEB3', 
'6570074VSEH2', 
'65700A2CSEE5', 
'65700A2WSEB1', 
'65700J26SEA4', 
'65700K70SEE0', 
'65700M9ZSEF3', 
'6A03K184SOG6', 
'6A05X064SOC1', 
'6A064270SOG7', 
'6A069064SOA7', 
'6A069071SOA0', 
'6A06C198SOE6', 
'6A06G050SOG6', 
'6A06G188SOB6', 
'6A06K030SOG5', 
'6A06L033SOF0', 
'6A06M050SOB5', 
'6A06N111SOC1', 
'6A06N266SOA6', 
'6A06Y073SOF6', 
'6A06Z074SOD0', 
'6A06Z076SOE1', 
'6A071192SOF0', 
'6A077029SOG3', 
'6A077094SOB7', 
'6A078207SOC2', 
'6S0AH195SOB7', 
'6570074JSEA4', 
'65700A2XSEF3', 
'65700AACSEF6', 
'65700C71SEA4', 
'65700G42SEC4', 
'65700J08SEA0', 
'65700J5JSED1', 
'6A03T394SOF4', 
'6A04M044SOA6', 
'6A061086SOA5', 
'6A066219SOE1', 
'6A06B288SOG3', 
'6A06G049SOD5', 
'6A06G334SOE6', 
'6A06J097SOE0', 
'6A06J098SOA7', 
'6A06K048SOA7', 
'6A06U064SOE0', 
'6A070392SOC7', 
'6A071024SOF0', 
'6A077174SOG6', 
'6A07B275SOG7', 
'65700M5XSEB0', 
'6A066175SOA2', 
'6A068057SOB3', 
'6A068063SOD5', 
'6A069043SOC3', 
'6A06B222SOF4', 
'6A06J095SOC7', 
'6A06L196SOH0', 
'6A06P295SOD3', 
'6A06X068SOA0', 
'6A070194SOF6', 
'6A074179SOE6', 
'6A077088SOH0', 
'6A07B180SOF6', 
'65700J10SEA0', 
'6A05T054SOD4', 
'6A068056SOE4', 
'6A069062SOH1', 
'6A06B040SOA3', 
'6A06I050SOA1', 
'6A06T208SOG5', 
'6A071012SOG6'
"""

SQLquery ="""
-- sql for MEASUREMENT retrieval 
select lot_Id, wafer_Id, normalized_TestX as chipX, normalized_TestY as chipY, family_Code, date, proc_Level as level, meas_Pd_Id as step, meas_Eqp_Id as tool, dcItem_Name as parameter, 
	 dcItem_Unit as units, 
	 dcItem_Value as parmValue,
	 kerf_X, kerf_Y, site_X_Coord as locX, site_Y_Coord as locY, area_Center_5, radius_Center_5, quadrant, locationType
 from DMIW.MeasuredRawFact measuredRawFact, 
	 DMIW_SYSTEMS.MeasuredParm measuredParm, 
	 DMIW_SYSTEMS.Geography geography, 
	 (select measuredWaferKey, lot_Id, meas_Wafer_Id as wafer_Id, family_Code, lot_Grade, measurement_Date as date, proc_Level, meas_pd_Id, meas_Eqp_Id 
	 from DMIW_SYSTEMS.MeasuredWafer measuredWafer 
	 where Measurement_date >= (current date - 300 days) and Tech_id = '5H 14HP' and Meas_pd_id like '%MTRCDRIEVETB5TTP.1' and Wafer_id in ({}) 
	 group by measuredWaferKey, lot_Id, meas_Wafer_Id, family_Code, lot_Grade, measurement_Date, proc_Level, meas_pd_Id, meas_Eqp_Id) measuredWafer 
 where measuredWafer.measuredWaferKey = measuredRawFact.measuredWaferKey and 
	 measuredParm.measuredParmKey = measuredRawFact.measuredParmKey and 
	 geography.geographyKey = measuredRawFact.geographyKey and 
	 meas_Pd_Id <> '' and dcItem_Value is not null and 
	 ucase(dcItem_Name) not in ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
 	 and ( ucase(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) like '%IHS_CD1%' )  
 order by lot_Id, wafer_Id, dcItem_Name, proc_Level 
 fetch first 1000000 rows only with ur 
;
""".format(waferids)

Data = pd.read_sql(SQLquery,con)

In [13]:
Data.to_csv("~/Downloads/SQLPULL.csv")